In [1]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False

from glob import glob
import re
import math

http://www.me.go.kr/home/web/policy_data/read.do?pagerOffset=10&maxPageItems=10&maxIndexPages=10&searchKey=title&searchValue=%ED%8F%90%EC%88%98%EC%B2%98%EB%A6%AC%EC%8B%9C%EC%84%A4&menuId=10263&orgCd=&condition.deleteYn=N&seq=7255
환경부 환경정책 물환경관리 공공폐수처리시설 현황

* 2018년 공공폐수처리시설 운영현황_ 공지용.xlsx -> 폐수처리시설_2018
* 2017년 공공폐수처리시설 운영현황(최종,게재)(1).xlsx -> 폐수처리시설_2017
* 2016년 공공폐수처리시설 운영현황_최종_공지용(17.12.31).xlsx -> 폐수처리시설_2016
* 2015년 폐수종말처리시설 운영실태조사표_최종_공지용.xlsx -> 폐수처리시설_2015

In [2]:
# 컬럼명이 멀티컬럼
# 사전에 컬럼명을 엑셀로 전처리
# 컬럼을 엑셀을 통해서 사전에 전처리
df = pd.read_excel('../../data/team/폐수처리시설_2015.xlsx')
col_list = df.columns

In [3]:
df1 = pd.read_excel('../../data/team/폐수처리시설_2015.xlsx', skiprows=3)
df2 = pd.read_excel('../../data/team/폐수처리시설_2016.xlsx', skiprows=3)
df3 = pd.read_excel('../../data/team/폐수처리시설_2017.xlsx', skiprows=3)
df4 = pd.read_excel('../../data/team/폐수처리시설_2018.xlsx', skiprows=3)

In [4]:
# 어째서인지 2016년 자료는 컬럼이 한개 마지막에 한개 추가되어 제거
df2 = df2.iloc[:,0:47]

In [5]:


df1.iloc[:,0] = 2015
df2.iloc[:,0] = 2016
df3.iloc[:,0] = 2017
df4.iloc[:,0] = 2018


In [6]:
a = pd.concat([df1,df2,df3,df4])
a.columns = col_list

In [7]:
a = a.rename(columns = {'구분':'년'})

In [8]:
a.to_csv('../../data/team/폐수처리시설_concat.csv',index=False, encoding ='utf-8')
a.to_csv('../../data/team/폐수처리시설_concat_R.csv',index=False, encoding ='ANSI')

In [9]:
# 시군이 환경부로 되어있는거 7개를 손으로 변경
df = pd.read_csv('../../data/team/폐수처리시설_concat_R.csv', encoding = 'ANSI')


# 제주 데이터가 없어서 밑 링크의 하수도통계, 하수처리 항목에서 해결

http://www.me.go.kr/home/web/policy_data/read.do?pagerOffset=0&maxPageItems=10&maxIndexPages=10&searchKey=title&searchValue=%ED%86%B5%EA%B3%84&menuId=10264&orgCd=&condition.deleteYn=N&seq=6937

* 전국하수도통계에서 공공하수처리시설 시트의 제주도데이터를 통해 제주시 데이터 생성
* 2015하수도통계(최종)
* 2016 하수도통계(최종)
* (최종) 2017년 하수도통계
* 2018년 하수도통계(하수도시설 현황)
* 주시 임시방편.csv

In [10]:
# 불필요한컬럼 2차로 제거
df1 = df.drop([ '유입BOD(mg/L)', '대권역명','유역청','대권역명','중권역명','소권역명','공동처리구역면적(ha)',
       '총처리량\n(천톤/년)','중권역명', '소권역명','방류BOD(mg/L)', '유입COD(mg/L)', '방류COD(mg/L)',
       '유입SS(mg/L)', '방류SS(mg/L)', '유입T-N(mg/L)', '방류T-N(mg/L)', '유입T-P(mg/L)',
       '방류T-P(mg/L)', '유입대장균군수(개/mL)', '방류대장균군수(개/mL)', '유입생태독성(TU)',
       '방류생태독성(TU)','평균\n처리효율\n(%).1','평균\n처리효율\n(%).2','평균\n처리효율\n(%).3',
         '평균\n처리효율\n(%).4','평균\n처리효율\n(%).5','평균\n처리효율\n(%).6','방류생태독성(TU)평균','유입생태독성(TU)평균'],axis = 1)

In [11]:
df1 = df1.dropna()

In [12]:
df1.to_csv('../../data/team/폐수처리시설_1.csv',index=False, encoding ='utf-8')
df1.to_csv('../../data/team/폐수처리시설_1_R.csv',index=False, encoding ='ANSI')
df1_jeju = pd.read_csv('../../data/team/제주시 임시방편.csv', encoding = 'ANSI')
df1_all= ''
df1_all = pd.concat([df1,df1_jeju])

In [13]:
df1_all.to_csv('../../data/team/폐수처리시설_all.csv',index=False, encoding ='utf-8')
df1_all.to_csv('../../data/team/폐수처리시설_all_R.csv',index=False, encoding ='ANSI')

# 해협별 분류가 필요

1. 시군을 기준으로 해엽을 분류, 년도별 평균값을 구합 df2 -해엽을 기준으로 1차 넣음
2. df_all의 구군을 추출, df1의 도로명 주소에서 같은 위치를 찾은 뒤 이들을 평균냄, 이를 2차로 넣음
3. 

시별 생태구역을 넣는 작업을 위해 폐수처리시설데이터의 시군 csv파일을 생성
규칙성이 없어서 직접 분류

* df_si = pd.DataFrame(columns=['시','생태구역'])
* si = df1_all['시군'].unique()
* for i in range(len(si)):
    df_si.loc[i] = [si[i],1]
    

* df_si.to_csv('tmp.csv',index=False, encoding='ANSI')

In [17]:
df1_all = pd.read_csv('../../data/team/폐수처리시설_all.csv')
df_si = pd.read_csv('tmp.csv', encoding = 'ANSI')
df_si = df_si.set_index('시')

In [18]:
# 생태구역별 평균
df1_all['생태구역'] = ''
for i in df1_all.index:
    for j in df_si.index:
        if df1_all.loc[i, '시군'] == j:
            df1_all.loc[i,'생태구역'] = df_si.loc[j,'생태구역']
            break

In [19]:
df1_all.to_csv('../../data/폐수데이터+생태구역.csv',encoding= 'ANSI',index=False)

In [21]:
# 조작편의를 위해서 멀티인덱스를 컬럼으로 변환
df2  = df1_all.groupby(['년','생태구역']).mean()

df2_2015 = df2.xs(2015).reset_index()
df2_2015['년'] = 2015
df2_2016= df2.xs(2016).reset_index()
df2_2016['년'] = 2016
df2_2017 = df2.xs(2017).reset_index()
df2_2017['년'] = 2017
df2_2018 = df2.xs(2018).reset_index()
df2_2018['년'] = 2018

df2_mean = pd.concat([df2_2015,df2_2016,df2_2017,df2_2018])


In [24]:
# 시군별 평균치

df3 = df1_all.groupby(['년','시군']).mean()

df3_2015 = df3.xs(2015).reset_index()
df3_2015['년'] = 2015
df3_2016= df3.xs(2016).reset_index()
df3_2016['년'] = 2016
df3_2017 = df3.xs(2017).reset_index()
df3_2017['년'] = 2017
df3_2018 = df3.xs(2018).reset_index()
df3_2018['년'] = 2018

df3_mean = pd.concat([df3_2015,df3_2016,df3_2017,df3_2018])


    
df3_mean.to_csv('../../data/team/시군별방류수.csv', index=False, encoding = 'utf-8')
df3_mean.to_csv('../../data/team/시군별방류수_R.csv', index=False, encoding = 'ANSI')

In [23]:
df2_mean.to_csv('../../data/team/생태구역별방류수.csv', index=False, encoding = 'utf-8')
df2_mean.to_csv('../../data/team/생태구역별방류수_R.csv', index=False, encoding = 'ANSI')

In [25]:
df_all = pd.read_csv('../../data/team/수질+수거량+정화.csv')
df2_mean = pd.read_csv('../../data/team/생태구역별방류수.csv')
df_all = df_all.drop('Ds', axis= 1)
df3_mean = pd.read_csv('../../data/team/시군별방류수.csv')
# 컬럼추가
list_a = list(df2_mean.columns[1:-1])
for i in list_a:
    df_all[i] = ''

In [26]:
# 생태구 통합
for i in df_all.index:
    area = df_all.loc[i,'생태구역']
    year = int(df_all.loc[i,'년월']/100)
    for j in df2_mean.index:
        if (df2_mean.loc[j,'생태구역'] == area)&(df2_mean.loc[j,'년'] == year):
            df_all.loc[i,'일별처리량':] = df2_mean.loc[j,'일별처리량':'방류대장균군수(개/mL)평균']
            

In [27]:
# 시, 구군통합
for i in df_all.index:
    area = df_all.loc[i,'구군']
    year = int(df_all.loc[i,'년월']/100)
    for j in df3_mean.index:
        if (df3_mean.loc[j,'시군'].find(area) != -1)&(df3_mean.loc[j,'년'] == year):
            df_all.loc[i,'일별처리량':] = df3_mean.loc[j,'일별처리량':'방류대장균군수(개/mL)평균']
            break
            

In [28]:
# 도로명주소, 구군통합
for i in df_all.index: 
    if df_all.loc[i,'구군'] != '동해':
        area = df_all.loc[i,'구군']
    else :
        area = '동해시'
        
    tmp = df1_all[df1_all['소재지\n(도로명 주소 및 지번주소 병기)'].str.contains(area)]
    year = int(df_all.loc[i,'년월']/100)
  
    if len(tmp) != 0:
        for j in tmp['년'].unique():
            if year == j:
                df_all.loc[i,'일별처리량':] = tmp[tmp['년'] == j].mean().iloc[1:]
                

In [32]:
df_all = df_all.drop(['연안명칭','St.\nNo\n','구군','유입BOD(mg/L)평균','유입COD(mg/L)평균','유입SS(mg/L)평균','유입T-N(mg/L)평균','유입T-P(mg/L)평균','시설용량(톤/일)(A)'], axis=1)

In [41]:
df_all.to_csv('../../data/team/수질데이터최종.csv', index=False, encoding = 'utf-8')
df_all.to_csv('../../data/team/수질데이터최종_R.csv', index=False, encoding = 'ANSI')